In [ ]:
!pip install flask
!pip install flask_ngrok
!pip install apex
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4
!apt-get install xpdf

In [ ]:
mkdir -p data/amazon

In [ ]:
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf

In [ ]:
ls data/amazon

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [6]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 40

In [20]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(scheme="https",host=HOST,port=9243, username=USER, password=PWD, index="document")

04/11/2021 12:51:05 - INFO - elasticsearch -   HEAD https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/ [status:200 request:0.857s]
04/11/2021 12:51:05 - INFO - elasticsearch -   HEAD https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/document [status:200 request:0.211s]
04/11/2021 12:51:05 - INFO - elasticsearch -   GET https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/document [status:200 request:0.214s]
04/11/2021 12:51:05 - INFO - elasticsearch -   PUT https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/document/_mapping [status:200 request:0.216s]
04/11/2021 12:51:06 - INFO - elasticsearch -   HEAD https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/label [status:200 request:0.214s]


In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [9]:

directory = "data"
import os
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.preprocessor.preprocessor import PreProcessor

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["de","en"])

processor = PreProcessor(clean_empty_lines=True,
                         clean_whitespace=True,
                         clean_header_footer=True,
                         split_by="word",
                         split_length=200,
                         split_respect_sentence_boundary=True)
docs = []
for filename in os.listdir(directory):

    # Run the conversion on each file (PDF -> 1x doc)
    d = converter.convert(os.path.join(directory, filename), meta={"name":filename})

    # clean and split each dict (1x doc -> multiple docs)
    d = processor.process(d)
    docs.extend(d)

# Let's have a look at the first 3 entries:
print(docs[:3])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


04/11/2021 12:36:47 - WARNING - haystack.preprocessor.preprocessor -   A sentence found with word count higher than the split length.


[{'text': "Table of Contents\n\nTitle Page\nCopyright Page\nDedication\nPreface\n\nCHAPTER I - Overview of the Industry\nCHAPTER II - Products That We Offer\nCHAPTER III - The Marketplaces and Order Management\nCHAPTER IV - The Trade-Processing System\nCHAPTER V - Margin\nCHAPTER VI - Settlement\nCHAPTER VII - Custodial Services\nCHAPTER VIII - Role of the Banks\nCHAPTER IX - Cash Accounting\nCHAPTER X - Brokerage Accounting\nCHAPTER XI - Stock Record\nCHAPTER XII - General Ledger\nCHAPTER XIII - Regulatory Reporting\nCHAPTER XIV - Conclusion\n\nGlossary\nIndex\n\nPORTFOLIO\nPublished by the Penguin Group\nPenguin Group (USA) Inc., 375 Hudson Street, New York, New York 10014, U.S.A.  Penguin Group\n(Canada), 90 Eglinton Avenue East, Suite 700, Toronto, Ontario, Canada M4P 2Y3 (a division of Pearson\nPenguin Canada Inc.)  Penguin Books Ltd, 80 Strand, London WC2R 0RL, England  Penguin Ireland, 25 St.\nStephen's Green, Dublin 2, Ireland (a division of Penguin Books Ltd)  Penguin Books Au

In [152]:
len(docs)

752

In [153]:
document_store.write_documents(docs)

04/11/2021 12:19:00 - INFO - elasticsearch -   POST https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/_bulk?refresh=wait_for [status:200 request:2.818s]
04/11/2021 12:19:02 - INFO - elasticsearch -   POST https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/_bulk?refresh=wait_for [status:200 request:1.797s]


In [21]:
 from haystack.retriever.dense import DensePassageRetriever
 retriever = DensePassageRetriever(document_store=document_store,
                                      query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                      passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                      max_seq_len_query=64,
                                      max_seq_len_passage=256,
                                      batch_size=2,
                                      use_gpu=True,
                                      embed_title=True,
                                      use_fast_tokenizers=True
                                      )

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [11]:
document_store.update_embeddings(retriever)

04/11/2021 12:37:50 - INFO - elasticsearch -   POST https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/document/_count [status:200 request:0.217s]
04/11/2021 12:37:50 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for all 752 docs ...
04/11/2021 12:37:52 - INFO - elasticsearch -   POST https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/document/_search?scroll=1d&size=10000 [status:200 request:1.524s]
04/11/2021 12:37:52 - INFO - elasticsearch -   POST https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/_search/scroll [status:200 request:0.218s]
04/11/2021 12:37:52 - INFO - elasticsearch -   DELETE https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/_search/scroll [status:200 request:0.218s]
Creating Embeddings: 100%|██████████| 376/376 [09:56<00:00,  1.59s/ Batches]
04/11/2021 12:47:57 - INFO - elasticsearch -   POST https://enterprise-search-deploym

In [12]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

04/11/2021 12:48:17 - INFO - farm.utils -   Using device: CPU 
04/11/2021 12:48:17 - INFO - farm.utils -   Number of GPUs: 0
04/11/2021 12:48:17 - INFO - farm.utils -   Distributed Training: False
04/11/2021 12:48:17 - INFO - farm.utils -   Automatic Mixed Precision: None
04/11/2021 12:48:17 - INFO - filelock -   Lock 139646644566480 acquired on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


04/11/2021 12:48:17 - INFO - filelock -   Lock 139646644566480 released on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock
04/11/2021 12:48:17 - INFO - filelock -   Lock 139646615614608 acquired on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


04/11/2021 12:48:29 - INFO - filelock -   Lock 139646615614608 released on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
04/11/2021 12:48:36 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='prediction_heads' was already logged with value='TextSimilarityHead' for run ID='11e783b2acd84ed9883b65407fc81c6b'. Attempted logging new value 'QuestionAnsweringHead'.
04/11/2021 12:48:37 - INFO - filelock -   Lock 139646641122512 acquired on /root/.cache/huggingface/transformers/81c80edb4c6cefa5cae64ccfdb34b3b309ecaf60da99da7cd1c17e24a5d36eb5.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock


04/11/2021 12:48:37 - INFO - filelock -   Lock 139646641122512 released on /root/.cache/huggingface/transformers/81c80edb4c6cefa5cae64ccfdb34b3b309ecaf60da99da7cd1c17e24a5d36eb5.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock
04/11/2021 12:48:37 - INFO - filelock -   Lock 139646640962640 acquired on /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


04/11/2021 12:48:37 - INFO - filelock -   Lock 139646640962640 released on /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


04/11/2021 12:48:37 - INFO - filelock -   Lock 139646714667984 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


04/11/2021 12:48:37 - INFO - filelock -   Lock 139646714667984 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
04/11/2021 12:48:37 - INFO - filelock -   Lock 139646714626576 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


04/11/2021 12:48:37 - INFO - filelock -   Lock 139646714626576 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


04/11/2021 12:48:37 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='processor' was already logged with value='TextSimilarityProcessor' for run ID='11e783b2acd84ed9883b65407fc81c6b'. Attempted logging new value 'SquadProcessor'.
04/11/2021 12:48:37 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
04/11/2021 12:48:37 - INFO - farm.utils -   Using device: CPU 
04/11/2021 12:48:37 - INFO - farm.utils -   Number of GPUs: 0
04/11/2021 12:48:37 - INFO - farm.utils -   Distributed Training: False
04/11/2021 12:48:37 - INFO - farm.utils -   Automatic Mixed Precision: None
04/11/2021 12:48:38 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
04/11/2021 12:48:38 - INFO - farm.infer -    0    0 
04/11/2021 12:48:38 - INFO - farm.infer -   /w\  /w\
04/11/2021 12:48:38 - INFO - farm.infer -   /'\  / \
04/11/2021 12:48:38 - INFO - farm.infer -     


In [22]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [23]:
prediction = pipe.run(query="Who is a counterparty?", top_k_retriever=5, top_k_reader=5)

04/11/2021 12:51:52 - INFO - elasticsearch -   POST https://enterprise-search-deployment-a65c03.es.ap-southeast-1.aws.found.io:9243/document/_search [status:200 request:0.428s]
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.58s/ Batches]


In [19]:
print_answers(prediction, details="minimal")

{   'answers': [   {   'answer': 'Knight and Knune',
                       'context': 'On\n'
                                  'the other side of the trade is another '
                                  "broker/dealer--let's say Knight and Knune, "
                                  'which sold\n'
                                  "the stock. If SF&R's trade is executed "
                                  'against the omn',
                       'document_id': 'ca3a3498-9f71-4992-903f-818e51c2f3e1',
                       'meta': {   '_split_id': 369,
                                   'name': 'David M. Weiss - After the Trade '
                                           'Is Made_ Processing Securities '
                                           'Transactions  -Portfolio Hardcover '
                                           '(2006).pdf'},
                       'offset_end': 83,
                       'offset_end_in_doc': 340,
                       'offset_start': 67,
             

In [28]:
import json
import os
import logging
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from flask import Flask, request, jsonify
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts
from haystack.reader.farm import FARMReader
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.file_converter.pdf import PDFToTextConverter
from haystack.retriever.dense import DensePassageRetriever
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.pipeline import ExtractiveQAPipeline
from werkzeug.utils import secure_filename

#application settings
app = Flask(__name__)
CORS(app)


In [29]:
@app.route('/query',methods=['GET', 'POST'])
def search():
    """Return the n answers."""

    question = request.get_json()
    question = question['questions']

    prediction = pipe.run(query=question[0], top_k_retriever=3, top_k_reader=3)
    answer = []
  
    for res in prediction['answers']:
        answer.append(res['answer'])

    result = {"results":[prediction]}
    return json.dumps(result)


In [30]:
@app.route('/file-upload', methods = ['GET', 'POST'])
def upload_file():
   if request.method == 'POST':
      f = request.files['file']
      f.save(secure_filename(f.filename))
      return 'File Uploaded Successfully'

In [31]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


04/11/2021 13:37:16 - INFO - werkzeug -    * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c2d5baf86896.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


04/11/2021 13:38:21 - INFO - werkzeug -   127.0.0.1 - - [11/Apr/2021 13:38:21] "POST /file-upload HTTP/1.1" 200 -
04/11/2021 13:43:13 - INFO - werkzeug -   127.0.0.1 - - [11/Apr/2021 13:43:13] "POST /file-upload HTTP/1.1" 200 -


In [ ]:
from haystack.connector import Crawler
    
crawler = Crawler()
# crawl Haystack docs, i.e. all pages that include haystack.deepset.ai/docs/
docs = crawler.crawl(urls=["https://haystack.deepset.ai/docs/latest/get_startedmd"],
    output_dir="crawled_files",
    filter_urls= ["haystack\.deepset\.ai\/docs\/"])